In [1]:
"""
Static Security Bechmark
This script is designed to measure the ability of static analysis tools to identify safe and unsafe code.
"""

import json
import os
import sys
import re
import time
import subprocess
import pandas as pd
from pathlib import Path
from llmScan import *

In [ ]:
# Define the benchmark test function with timing
def test_vulnerabilities_benchmark(python_code_path):
    """
    Analyzes a Python file for security vulnerabilities

    Args:
        python_code_path (str): Path to the Python file to analyze

    Returns:
        tuple: (success, result, tool_name, scan_time)
        - success (bool): True if analysis successful, False otherwise
        - result (bool): True if vulnerable, False if safe
        - tool_name (str): Name of the static tool used
        - scan_time (float): Time taken for the scan in seconds
    """

    # Start timing
    start_time = time.time()

    # Define the Bandit command
    command = [
        'bandit',
        '-r', python_code_path,
        '--format', 'json',
        '-o', 'bandit_results.json',
        '-ll', '-iii'
    ]

    # Run the Bandit command
    try:
        subprocess.run(command, check=True)
        print("Bandit analysis completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running Bandit: {e}")
        return False, None, "Bandit", time.time() - start_time


    
